In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
df_movies = pd.read_csv('/content/drive/MyDrive/movie_recommendation/movies.csv')
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
df_movies.drop(columns = ['title'], inplace =True)

In [4]:
df_movies.columns

Index(['movieId', 'genres'], dtype='object')

In [5]:
df_movies.shape

(9742, 2)

In [6]:
df_links = pd.read_csv('/content/drive/MyDrive/movie_recommendation/links.csv')
df_links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [7]:
df_links.shape

(9742, 3)

In [8]:
df_link_movies = pd.merge( df_movies , df_links , on= 'movieId')
df_link_movies.head()

,movieId,genres,imdbId,tmdbId
0,1,Adventure|Animation|Children|Comedy|Fantasy,114709,862.0
1,2,Adventure|Children|Fantasy,113497,8844.0
2,3,Comedy|Romance,113228,15602.0
3,4,Comedy|Drama|Romance,114885,31357.0
4,5,Comedy,113041,11862.0


In [9]:
df_link_movies = df_link_movies.drop(columns =['imdbId'])

In [10]:
df_meta = pd.read_csv('/content/drive/MyDrive/movie_recommendation/movies_metadata.csv')
df_meta.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [11]:
df_meta.shape

(45466, 24)

In [12]:
df_meta.columns

Index(['adult', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'id',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'video',
       'vote_average', 'vote_count'],
      dtype='object')

In [13]:
df_meta = df_meta.drop(columns = [ 
    'belongs_to_collection', 'budget', 'genres', 'homepage', 'poster_path', 
    'production_companies', 'release_date', 'revenue', 'runtime', 'spoken_languages', 'title', 
    'video', 'vote_average', 'vote_count', 'popularity', 'production_countries', 'status', 'imdb_id'
    ])
df_meta.head()

,adult,id,original_language,original_title,overview,tagline
0,False,862,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",NaN
1,False,8844,en,Jumanji,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!
2,False,15602,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...
3,False,31357,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...
4,False,11862,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...


In [14]:
df_meta.rename(columns = {'id':'tmdbId'}, inplace = True)

In [15]:

for i in range(len(df_meta)):
    try:
        df_meta.loc[i, 'tmdbId'] = int(df_meta.loc[i, 'tmdbId'])
    except:
        print(i, df_meta.loc[i, 'tmdbId'])

19730 1997-08-20
29503 2012-09-29
35587 2014-01-01


In [16]:
df_meta =df_meta.drop([19730, 29503 , 35587])


In [17]:
df_meta_link_movies = pd.merge(df_link_movies, df_meta, on ='tmdbId')
df_meta_link_movies.head()

,movieId,genres,tmdbId,adult,original_language,original_title,overview,tagline
0,1,Adventure|Animation|Children|Comedy|Fantasy,862.0,False,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",NaN
1,2,Adventure|Children|Fantasy,8844.0,False,en,Jumanji,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!
2,3,Comedy|Romance,15602.0,False,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...
3,4,Comedy|Drama|Romance,31357.0,False,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...
4,5,Comedy,11862.0,False,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...


In [18]:
df = df_meta_link_movies

In [19]:
df.genres = df.genres.fillna('')
df.overview = df.overview.fillna('')
df.tagline = df.tagline.fillna('')
df.original_language = df.original_language.fillna('')

In [20]:
df.head()

,movieId,genres,tmdbId,adult,original_language,original_title,overview,tagline
0,1,Adventure|Animation|Children|Comedy|Fantasy,862.0,False,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",
1,2,Adventure|Children|Fantasy,8844.0,False,en,Jumanji,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!
2,3,Comedy|Romance,15602.0,False,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...
3,4,Comedy|Drama|Romance,31357.0,False,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...
4,5,Comedy,11862.0,False,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...


In [21]:
df['genres'] = df['genres'].replace( "\|", " " , regex=True) # replace "|" with <space>
df.head()

,movieId,genres,tmdbId,adult,original_language,original_title,overview,tagline
0,1,Adventure Animation Children Comedy Fantasy,862.0,False,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",
1,2,Adventure Children Fantasy,8844.0,False,en,Jumanji,When siblings Judy and Peter discover an encha...,Roll the dice and unleash the excitement!
2,3,Comedy Romance,15602.0,False,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,Still Yelling. Still Fighting. Still Ready for...
3,4,Comedy Drama Romance,31357.0,False,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",Friends are the people who let you be yourself...
4,5,Comedy,11862.0,False,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,Just When His World Is Back To Normal... He's ...


##Deleting duplicate movies

In [22]:
df['original_title'].duplicated().value_counts()

False    9266
True      289
Name: original_title, dtype: int64

In [23]:
duplicate_indices = df['original_title'][df['original_title'].duplicated()==True].index

for idx in duplicate_indices:
  df.drop(idx, inplace = True)

In [24]:
df.reset_index(drop = True, inplace =True)   #reset the index  #drop =true means dropping the old index

In [25]:
df.duplicated().value_counts()

False    9266
dtype: int64

In [26]:
df['description'] =  df.overview +" " + df.tagline +" "  + df.genres  +" "  + df.original_language
df.description = df.description.fillna('')
df_cleaned = df.drop(columns= ['overview', 'tagline' , 'genres', 'adult', 'original_language'])
df_cleaned

,movieId,tmdbId,original_title,description
0,1,862.0,Toy Story,"Led by Woody, Andy's toys live happily in his ..."
1,2,8844.0,Jumanji,When siblings Judy and Peter discover an encha...
2,3,15602.0,Grumpier Old Men,A family wedding reignites the ancient feud be...
3,4,31357.0,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom..."
4,5,11862.0,Father of the Bride Part II,Just when George Banks has recovered from his ...
...,...,...,...,...
9261,175705,7014.0,Themroc,"Made without proper language, just gibberish a..."
9262,175707,411516.0,Ein deutsches Leben,"Brunhilde Pomsel describes herself as an ""apol..."
9263,175743,433410.0,Selbstkritik eines bürgerlichen Hundes,A bourgeois dog confesses how he was transform...
9264,175781,28469.0,Der Herr Karl,No overview found. Comedy de


In [27]:
df_cleaned.shape

(9266, 4)

In [28]:
df_cleaned.description[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene. Afraid of losing his place in Andy's heart, Woody plots against Buzz. But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences.  Adventure Animation Children Comedy Fantasy en"

In [29]:
df_cleaned['description'] = df_cleaned['description'].replace("\.", ' ', regex =True) #replacing dots with <space>

In [30]:
df_cleaned.description[0]

"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear onto the scene  Afraid of losing his place in Andy's heart, Woody plots against Buzz  But when circumstances separate Buzz and Woody from their owner, the duo eventually learns to put aside their differences   Adventure Animation Children Comedy Fantasy en"

##Remove punctuations

In [31]:
import string
string.punctuation 

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [32]:
exclude = string.punctuation 

def remove_punc(text):
  return text.translate(str.maketrans('','', exclude))
  

In [33]:
df_cleaned['description'] = df_cleaned['description'].apply(remove_punc)

In [34]:
df_cleaned.description[0]

'Led by Woody Andys toys live happily in his room until Andys birthday brings Buzz Lightyear onto the scene  Afraid of losing his place in Andys heart Woody plots against Buzz  But when circumstances separate Buzz and Woody from their owner the duo eventually learns to put aside their differences   Adventure Animation Children Comedy Fantasy en'

In [35]:
titles = df_cleaned.original_title
titles

0                                    Toy Story
1                                      Jumanji
2                             Grumpier Old Men
3                            Waiting to Exhale
4                  Father of the Bride Part II
                         ...                  
9261                                   Themroc
9262                       Ein deutsches Leben
9263    Selbstkritik eines bürgerlichen Hundes
9264                             Der Herr Karl
9265     LEGO DC Super Hero Girls: Brain Drain
Name: original_title, Length: 9266, dtype: object

In [36]:
indices = pd.Series( titles.index, index = titles)
indices

original_title
Toy Story                                    0
Jumanji                                      1
Grumpier Old Men                             2
Waiting to Exhale                            3
Father of the Bride Part II                  4
                                          ... 
Themroc                                   9261
Ein deutsches Leben                       9262
Selbstkritik eines bürgerlichen Hundes    9263
Der Herr Karl                             9264
LEGO DC Super Hero Girls: Brain Drain     9265
Length: 9266, dtype: int64

#Cosine Similarity

In [37]:
tf = TfidfVectorizer(
    max_features=None, analyzer='word', token_pattern=r'\w{1,}', 
    ngram_range=(1, 3), min_df=0, 
    stop_words='english',
    lowercase=True
    )

tfidf_matrix = tf.fit_transform(df_cleaned['description'])

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [38]:
cosine_sim[0]

array([1.00000000e+00, 1.03420883e-02, 6.14381519e-04, ...,
       1.48889853e-03, 3.53429517e-03, 1.91951140e-03])

#Recommedation function

In [39]:
def recommender(title):

  if df.original_title.str.contains(title).value_counts().shape[0]==1: 
    print('The Movie is not in the data set')
  
  else:  
    #find index of the given movie
    idx = indices[title] 

    #find pairwise cosine similarity values
    cosine_sim_scores = list(enumerate(cosine_sim[idx])) 

    #sort based on the cosine similarity scores
    cosine_sim_scores_sorted = sorted(cosine_sim_scores , key = lambda x: x[1] , reverse = True)

    #select top 10  movies similar to given movie
    cosine_sim_scores_sorted_top_10 = cosine_sim_scores_sorted[1:11]

    #find indices of the top 10 movies 
    cosine_top_10_indices = [i[0] for i in cosine_sim_scores_sorted_top_10 ]

    #find the top 10 most similar movies
    cosine_movie_list = df_cleaned['original_title'].iloc[cosine_top_10_indices]
   
    movies_rec = pd.DataFrame(cosine_movie_list)

  
    return movies_rec

##Input the movie name by running the below code

In [40]:
movie_name = str(input('Type the movie name and press Enter:'))
print('Recommended movies for {} are as follows'.format(movie_name))
recommender(movie_name)

Type the movie name and press Enter:Iron Man
Recommended movies for Iron Man are as follows


,original_title
7123,Iron Man 2
8414,Avengers: Age of Ultron
7897,The Invincible Iron Man
8420,Guardians of the Galaxy Vol. 2
8671,Robot Overlords
2731,Mad Max
8746,Turbo Kid
8213,Guardians of the Galaxy
2916,The Time Machine
8513,Clown
